In [1]:
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Dense, Flatten
import numpy as np
import glob
import os

import pickle
import time

Using TensorFlow backend.


In [2]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

In [3]:
joined = bbox.join(cat,lsuffix='_',rsuffix='')

In [4]:
joined.head()

,image_name_,x_1,y_1,x_2,y_2,image_name,category_label
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3


In [5]:
labels = joined['category_label'].value_counts()
top_labels = labels[labels>1000].index.values

In [6]:
subset = pd.DataFrame([], columns=joined.columns)
for l in top_labels:
    has_label = joined[joined['category_label']==l]
    sample = np.random.randint(has_label.shape[0],size=1000)
    subset = subset.append(has_label.iloc[sample])

In [7]:
subset['category_label'] = pd.Categorical(subset['category_label'])
subset['code'] = subset['category_label'].cat.codes

In [8]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

def crop_resize_DF(bbox):
    img = PIL.Image.open('./Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [9]:
def generator(df, batch_size):
    while True:
        subset = np.random.randint(df.shape[0],size=batch_size)
        proc_imgs = []
        for i in subset:
            proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
    
        attr = df.iloc[subset]['code'].values
        yield np.array(proc_imgs), attr

In [10]:
def gen_batch(df,batch_size):
    subset = np.random.randint(df.shape[0],size=batch_size)
    proc_imgs = []
    for i in subset:
        proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
        print(df.iloc[i]['image_name'],df.iloc[i]['code'])
    
    attr = df.iloc[subset]['code'].values
    return np.array(proc_imgs), attr

In [11]:
attr = subset['category_label'].values

In [12]:
np.bincount(attr)

array([   0,    0, 1000, 1000,    0,    0, 1000,    0,    0,    0, 1000,
       1000,    0,    0,    0,    0, 1000, 1000, 1000, 1000,    0,    0,
          0,    0, 1000,    0, 1000,    0,    0, 1000, 1000,    0, 1000,
       1000, 1000, 1000,    0,    0,    0, 1000,    0, 1000, 1000,    0,
       1000,    0,    0,    0, 1000])

In [13]:
(classes,counts) = np.unique(attr,return_counts=True)

In [14]:
classes

array([ 2,  3,  6, 10, 11, 16, 17, 18, 19, 24, 26, 29, 30, 32, 33, 34, 35,
       39, 41, 42, 44, 48])

In [15]:
class_weights = joined.shape[0]/(counts*len(classes))

In [16]:
class_weights

array([ 13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455])

In [17]:
from keras import optimizers
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
l = 0.01

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output

x = Flatten(name='flatten')(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

x = Dense(4096, activation='relu', name='fc1', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

predictions = Dense(len(classes), activation='softmax',name='predictions', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)

new_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


new_model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy',
                 metrics = ['sparse_categorical_accuracy'])

In [18]:
new_model.layers[-1].output

<tf.Tensor 'predictions/Softmax:0' shape=(?, 22) dtype=float32>

In [19]:
from keras import callbacks
tbCB = callbacks.TensorBoard(log_dir="logs/DF_category{}".format(time.time()), histogram_freq=0, write_graph=False, write_images=False)
checkpointCB = callbacks.ModelCheckpoint("checkpoints/DF_category{}".format(time.time()))

h = new_model.fit_generator(generator(subset,128),steps_per_epoch=70,epochs=80,
                            validation_data=generator(subset,128),validation_steps=1,
                            verbose=1,callbacks=[tbCB,checkpointCB])

Epoch 1/80
70/70 [==============================] - 168s - loss: 46.2200 - sparse_categorical_accuracy: 0.5232 - val_loss: 36.7868 - val_sparse_categorical_accuracy: 0.6797
Epoch 2/80
70/70 [==============================] - 166s - loss: 28.9581 - sparse_categorical_accuracy: 0.7007 - val_loss: 23.0566 - val_sparse_categorical_accuracy: 0.6797
Epoch 3/80
70/70 [==============================] - 167s - loss: 17.6102 - sparse_categorical_accuracy: 0.7646 - val_loss: 13.2453 - val_sparse_categorical_accuracy: 0.8516
Epoch 4/80
70/70 [==============================] - 166s - loss: 10.8468 - sparse_categorical_accuracy: 0.7977 - val_loss: 8.3385 - val_sparse_categorical_accuracy: 0.8203
Epoch 5/80
70/70 [==============================] - 169s - loss: 7.2849 - sparse_categorical_accuracy: 0.7902 - val_loss: 6.3177 - val_sparse_categorical_accuracy: 0.7344
Epoch 6/80
70/70 [==============================] - 166s - loss: 5.5524 - sparse_categorical_accuracy: 0.7942 - val_loss: 4.8611 - val_spa

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224]
	 [[Node: block1_conv2/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](block1_conv1/Relu, block1_conv2/kernel/read)]]
	 [[Node: Mean_3/_157 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_220_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'block1_conv2/convolution', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/d

In [26]:
h = new_model.fit_generator(generator(subset,128),steps_per_epoch=70,epochs=80,
                            validation_data=generator(subset,128),validation_steps=1,
                            verbose=1,callbacks=[tbCB,checkpointCB])

Epoch 1/80
70/70 [==============================] - 166s - loss: 4.5185 - sparse_categorical_accuracy: 0.7779 - val_loss: 3.7862 - val_sparse_categorical_accuracy: 0.8594
Epoch 2/80
70/70 [==============================] - 165s - loss: 3.4771 - sparse_categorical_accuracy: 0.8317 - val_loss: 3.1831 - val_sparse_categorical_accuracy: 0.8125
Epoch 3/80
70/70 [==============================] - 168s - loss: 3.1164 - sparse_categorical_accuracy: 0.8247 - val_loss: 3.0853 - val_sparse_categorical_accuracy: 0.8359
Epoch 4/80
70/70 [==============================] - 164s - loss: 3.0080 - sparse_categorical_accuracy: 0.8223 - val_loss: 3.0063 - val_sparse_categorical_accuracy: 0.8047
Epoch 5/80
52/70 [=====================>........] - ETA: 38s - loss: 2.9539 - sparse_categorical_accuracy: 0.8155

KeyboardInterrupt: 

In [29]:
new_model.save_weights('DF-category-retrain.h5')

In [ ]:
p = new_model.predict_generator(generator(subset,64),steps=1)

In [ ]:
p

In [ ]:
plt.hist()

In [ ]:
np.argmax(p,axis=1)

In [ ]:
counts[17]/np.sum(counts)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.bar(np.arange(len(counts)),counts)

In [20]:
X, Y = gen_batch(subset,64)

img/Abstract_Print_Crop_Top/img_00000059.jpg 8
img/Trouble_Graphic_Sweater/img_00000045.jpg 5
img/Floral-Embroidered_Kimono/img_00000009.jpg 20
img/Hooded_High-Neck_Jacket/img_00000062.jpg 4
img/Light_Wash_-_Skinny_Jeans/img_00000004.jpg 10
img/Tribal_Print_Fit_&_Flare_Dress/img_00000036.jpg 18
img/Southwestern_Print_Blouse/img_00000023.jpg 1
img/Dolman_Sleeve_Hoodie/img_00000064.jpg 3
img/Open-Front_Multi-Stripe_Cardigan/img_00000024.jpg 2
img/Diamond_Open-Knit_Cardigan/img_00000024.jpg 2
img/Tribal_Print_Leggings/img_00000044.jpg 12
img/Boxy_Embroidered_Top/img_00000024.jpg 8
img/Abstract_Print_Capri_Leggings/img_00000020.jpg 12
img/Faux_Snakeskin-Trimmed_Blazer/img_00000065.jpg 0
img/Lace-Trimmed_Floral_Shorts/img_00000046.jpg 13
img/Marled_Colorblock_Raglan_Hoodie/img_00000015.jpg 3
img/Strapless_Wide-Leg_Jumpsuit/img_00000002.jpg 19
img/Abstract_Print_Surplice_Top/img_00000022.jpg 8
img/Loop_Knit_Open-Front_Cardigan/img_00000038.jpg 2
img/Striped_Mini_Dress/img_00000008.jpg 18
img

In [21]:
p = new_model.predict(X)

In [22]:
np.argmax(p,axis=1)

array([ 8,  5, 20,  4, 10, 18,  1,  3,  5,  2, 12,  1, 12,  0, 13,  3, 19,
        8,  2, 18,  5, 18,  1, 20, 18, 18, 13,  6,  7, 12, 18, 14, 15, 19,
       13, 13,  7,  6,  9,  4,  7, 13, 17,  2,  0,  9,  0, 21,  0, 15, 11,
        4,  8,  6, 18,  9, 10, 18, 13, 19, 19,  5,  0, 11])

In [23]:
Y

array([ 8,  5, 20,  4, 10, 18,  1,  3,  2,  2, 12,  8, 12,  0, 13,  3, 19,
        8,  2, 18,  5, 18,  1, 20, 18, 18,  9,  6,  7, 12, 18, 14, 15, 19,
       13, 13,  7,  6,  9,  4,  7, 13, 17,  2,  0,  9,  0, 19,  0, 15, 11,
        4,  8, 21, 18, 13, 15, 19, 13, 19, 19,  5,  0, 11], dtype=int8)

In [24]:
np.sum(np.argmax(p,axis=1)==Y)

56

In [ ]:
X[0]

In [ ]:
subset